In [1]:
# Get news title and paragraph text

In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [3]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Corey Anderson\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


<h2> Step 1: Scraping </h2>
<h3> NASA Mars News </h3>
<p> Get latest title and news story.</p>

In [5]:
mars_url = "https://redplanetscience.com/"

In [6]:
browser.visit(mars_url)

In [7]:
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
latest_titles = soup.find_all("div", class_ = "content_title")

In [9]:
print(latest_titles)

[<div class="content_title">Two Rovers to Roll on Mars Again: Curiosity and Mars 2020</div>, <div class="content_title">NASA Engineers Checking InSight's Weather Sensors</div>, <div class="content_title">Hear Audio From NASA's Perseverance As It Travels Through Deep Space</div>, <div class="content_title">NASA's New Mars Rover Is Ready for Space Lasers</div>, <div class="content_title">Screening Soon: 'The Pathfinders' Trains Lens on Mars</div>, <div class="content_title">NASA's Mars 2020 Rover Tests Descent-Stage Separation</div>, <div class="content_title">A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes</div>, <div class="content_title">NASA Moves Forward With Campaign to Return Mars Samples to Earth</div>, <div class="content_title">Mars Helicopter Attached to NASA's Perseverance Rover</div>, <div class="content_title">NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause</div>, <div class="content_title">Common Questions about InSight's 'Mole'</div>, <div class="con

In [10]:
news_title = latest_titles[0].text
print(news_title)

Two Rovers to Roll on Mars Again: Curiosity and Mars 2020


In [11]:
latest_p = soup.find_all("div", class_ = "article_teaser_body")
news_p  = latest_p[0].text
print(news_p)

They look like twins. But under the hood, the rover currently exploring the Red Planet and the one launching there this summer have distinct science tools and roles to play.


<h3> JPL Mars Space Images - Featured Image </h3>

In [12]:
pic_url = "https://spaceimages-mars.com/#"
browser.visit(pic_url)
html = browser.html
soup = bs(html, 'html.parser')

In [13]:
# feature_image_url

test = soup.find_all("img", class_ ="headerimage fade-in")
print(test)

url_end = test[0]["src"]
feature_image_url = f"https://spaceimages-mars.com/{url_end}"

print(feature_image_url)

[<img class="headerimage fade-in" src="image/featured/mars3.jpg"/>]
https://spaceimages-mars.com/image/featured/mars3.jpg


<h3> Mars Facts </h3>.

In [14]:
space_url = 'https://galaxyfacts-mars.com/'

In [15]:
browser.visit(space_url)
html = browser.html
soup = bs(html, 'html.parser')

In [16]:
mars_table = soup.find("table") # If you want the raw table from the site

In [17]:
# Now using pandas
easy_table = pd.read_html(space_url, header = 0)
mars_df = easy_table[0]
mars_df.set_index("Mars - Earth Comparison" , drop = True, inplace = True)

In [18]:
display(mars_df)

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [19]:
table_html = mars_df.to_html(index = False)

In [20]:
table_html = table_html.replace("\n", "")

In [21]:
print(table_html)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <td>2</td>      <td>1</td>    </tr>    <tr>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>


<h3> Mars Hemispheres </h3>

<p> Gather image urls.</p>

In [22]:
hemi_url = "https://marshemispheres.com/"

In [23]:
browser.visit(hemi_url)
html = browser.html
soup = bs(html, 'html.parser')

In [24]:
soup

<html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href="#" style="float:

In [25]:
descriptions = soup.find_all("div", class_ = "description")
print(descriptions)

[<div class="description">
<a class="itemLink product-item" href="cerberus.html">
<h3>Cerberus Hemisphere Enhanced</h3>
</a>
<span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/>
<p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p>
</div>, <div class="description">
<a class="itemLink product-item" href="schiaparelli.html">
<h3>Schiaparelli Hemisphere Enhanced</h3>
</a>
<span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/>
<p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p>
</div>, <div class="description">
<a class="itemLink product-item" href="syrtis.ht

In [26]:
images = soup.find_all("img", class_ = "thumb")
print(images)

[<img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/>, <img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/>, <img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/>, <img alt="Valles Marineris Hemisphere Enhanced thumbnail" class="thumb" src="images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png"/>]


In [27]:
images[0]["src"]

'images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'

In [28]:
urls = []
for pic in range(len(images)):
    urls.append(images[pic]["src"])

In [29]:
print(urls)

['images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png', 'images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png', 'images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png', 'images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png']


In [35]:
urls_final = []
for url in range(len(urls)):
    urls_final.append(f"https://marshemispheres.com/{urls[url]}")

In [36]:
print(urls_final)

['https://marshemispheres.com/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png', 'https://marshemispheres.com/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png', 'https://marshemispheres.com/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png', 'https://marshemispheres.com/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png']


In [37]:
pic_names = []

for j in range(len(descriptions)):
    pic_names.append(descriptions[j].h3.text.strip())

print(pic_names)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']


In [38]:
hemi = [None] * 4

In [39]:
for k in range(len(pic_names)):
    hemi[k] = {"title":pic_names[k], "img_url":urls_final[k]}

print(hemi)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png'}]


In [40]:
browser.quit()